In [1]:
import torch
import torchvision
from mqbench.prepare_by_platform import prepare_by_platform, BackendType
from mqbench.utils.state import enable_calibration, enable_quantization

In [3]:
model_to_quantize = torchvision.models.__dict__['resnet18'](pretrained=True)
model_to_quantize.cpu().eval()
dummy_input = torch.randn(1, 3, 224, 224, device='cpu')
model_prepared = prepare_by_platform(model_to_quantize, BackendType.Tensorrt)

[MQBENCH] INFO: Quantize model Scheme: BackendType.Tensorrt Mode: Eval
[MQBENCH] INFO: Weight Qconfig:
    FakeQuantize: LearnableFakeQuantize Params: {}
    Oberver:      MinMaxObserver Params: Symmetric: True / Bitwidth: 8 / Per channel: True / Pot scale: False / Extra kwargs: {}
[MQBENCH] INFO: Activation Qconfig:
    FakeQuantize: LearnableFakeQuantize Params: {}
    Oberver:      EMAMinMaxObserver Params: Symmetric: True / Bitwidth: 8 / Per channel: False / Pot scale: False / Extra kwargs: {}
[MQBENCH] INFO: Replace module to qat module.
[MQBENCH] INFO: Insert act quant x_post_act_fake_quantizer
[MQBENCH] INFO: Insert act quant conv1_post_act_fake_quantizer
[MQBENCH] INFO: Insert act quant maxpool_post_act_fake_quantizer
[MQBENCH] INFO: Insert act quant layer1_0_conv1_post_act_fake_quantizer
[MQBENCH] INFO: Insert act quant layer1_0_conv2_post_act_fake_quantizer
[MQBENCH] INFO: Insert act quant layer1_0_relu_1_post_act_fake_quantizer
[MQBENCH] INFO: Insert act quant layer1_1_conv1

In [4]:
enable_calibration(model_prepared)
model_prepared(dummy_input)
enable_quantization(model_prepared)

[MQBENCH] INFO: Enable observer and Disable quantize.
[MQBENCH] INFO: Disable observer and Enable quantize.


In [6]:
print(model_prepared)

GraphModule(
  (conv1): ConvReLU2d(
    3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3)
    (weight_fake_quant): LearnableFakeQuantize(
      fake_quant_enabled=tensor([1], dtype=torch.uint8), observer_enabled=tensor([0], dtype=torch.uint8), quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, ch_axis=0, scale=List[torch.Size([64])], zero_point=List
      (activation_post_process): MinMaxObserver(min_val=List, max_val=List ch_axis=0 pot=False)
    )
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Module(
    (0): Module(
      (conv1): ConvReLU2d(
        64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
        (weight_fake_quant): LearnableFakeQuantize(
          fake_quant_enabled=tensor([1], dtype=torch.uint8), observer_enabled=tensor([0], dtype=torch.uint8), quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_channel_symmetric, ch_axis=0, scale=List[torch.Size

In [12]:
model_prepared.graph.print_tabular()

opcode         name                                           target                                                      args                                                                                     kwargs
-------------  ---------------------------------------------  ----------------------------------------------------------  ---------------------------------------------------------------------------------------  --------
placeholder    x                                              x                                                           ()                                                                                       {}
call_module    x_post_act_fake_quantizer                      x_post_act_fake_quantizer                                   (x,)                                                                                     {}
call_module    conv1                                          conv1                                                       (x_post_act_